In [1]:
import requests
import telebot
from datetime import datetime
import json
from telebot import types

TOKEN = "7048352222:AAGSfTYD2PnCEX80xvnC5v5TLwOxsK3XGNw"
GIPHY_TOKEN = "aTBNLVPxfIp8xbsXhO8TrXhcYEXyRd2n"


HELP = '''
Список команд:
/start - первый запуск
/help - основные команды
/budget - начать выбор пива с бюджета
/taste - начать выбор пива с предпочтений по вкусу
/color - начать выбор пива с цвета
/snacks - подобрать подходящую закуску к пиву
/beer_gif - насладиться гифкой пива
У бота можно спросить:
"Как дела?"
"Сколько времени?"

А также поприветствовать его:
"Привет"
'''

bot = telebot.TeleBot(TOKEN)


def get_gif_by_name(name):

    url = "http://api.giphy.com/v1/gifs/search"

    param = {
        "api_key": GIPHY_TOKEN,
        "rating": "g",
        "q": name,
        "limit": 1,
        "lang": "ru"
    }

    result = requests.get(url, params=param)
    result_dict = result.json()
    # print(json.dumps(result_dict, sort_keys=True, indent=4))
    link_origin = result_dict["data"][0]["images"]["original"]["url"]

    return link_origin

@bot.message_handler(commands=['start'])
def start_handler(message):
    markup = types.ReplyKeyboardMarkup(one_time_keyboard=True, row_width=2, resize_keyboard=True)

    btn_pohvala = types.KeyboardButton('Похвали пиво')

    btn_help = types.KeyboardButton('Покажи, что умеешь')

    markup.row(btn_help)
    markup.row(btn_pohvala)

    bot.send_message(message.from_user.id, f"Привет,{message.from_user.first_name}! Я твой бот!", reply_markup=markup)


@bot.message_handler(func=lambda message: message.text == 'Покажи, что умеешь')
def help_handler(message):
    markup = types.ReplyKeyboardMarkup(one_time_keyboard=True, row_width=2)

    bot.send_message(message.from_user.id, HELP, reply_markup=markup)


@bot.message_handler(commands=['help'])
def help_handler(message):
    markup = types.ReplyKeyboardMarkup(one_time_keyboard=True, row_width=2)

    bot.send_message(message.from_user.id, HELP, reply_markup=markup)


@bot.message_handler(commands=['taste'])
def taste_handler(message):
    markup = types.ReplyKeyboardMarkup(one_time_keyboard=True, row_width=2, resize_keyboard=True)

    btn_kisloe = types.KeyboardButton('хочу кислое пиво')
    btn_sladkoe = types.KeyboardButton('хочу сладкое пиво')
    btn_terpkoe = types.KeyboardButton('хочу горькое пиво')

    markup.row(btn_terpkoe)
    markup.row(btn_sladkoe, btn_kisloe)

    bot.send_message(message.from_user.id, 'Выберите тип пива', reply_markup=markup)


@bot.message_handler(commands=['color'])
def color_handler(message):
    markup = types.ReplyKeyboardMarkup(one_time_keyboard=True, row_width=2, resize_keyboard=True)

    btn_light = types.KeyboardButton('хочу светлое пиво')
    btn_dark = types.KeyboardButton('хочу темное пиво')

    markup.row(btn_light)
    markup.row(btn_dark)

    bot.send_message(message.from_user.id, 'Выберите тип пива', reply_markup=markup)


@bot.message_handler(commands=['budget'])
def money_handler(message):
    markup = types.ReplyKeyboardMarkup(one_time_keyboard=True, row_width=2)
    btn_b1 = types.KeyboardButton('40 - 50 рублей')
    btn_b2 = types.KeyboardButton('50 - 300 рублей')
    btn_b3 = types.KeyboardButton('> 300 рублей')
    markup.row(btn_b1)
    markup.row(btn_b2, btn_b3)

    bot.send_message(message.from_user.id, 'Какой у вас бюджет?', reply_markup=markup)


@bot.message_handler(commands=['snacks'])
def snacks_link_handler(message):
    bot.send_message(message.from_user.id,
                     "Хочешь подобрать еще и подходящую закуску к напитку богов? Заходи вот в этого бота!\nhttps://t.me/Liquid_gold_snacks_bot")


@bot.message_handler(commands=['beer_gif'])
def gif_handler(message):
    bot.send_message(message.from_user.id, "Напиши слово пиво))))")
    bot.register_next_step_handler(message, get_find_name)


def get_find_name(message):
    bot.send_message(message.from_user.id, f"Эт ты))))))))")
    link = get_gif_by_name(message.text)
    bot.send_animation(message.from_user.id, link)


@bot.message_handler(content_types=['text'])
def text_handler(message):
    text = message.text.lower()
    if text == "хочу светлое пиво":
        bot.send_message(message.chat.id, 'Для вас сударь посоветую Blanc, Peroni, Argus.')
    elif text == "хочу темное пиво":
        bot.send_message(message.chat.id, 'Привет гурман, предлагаю тебе темный Kozel, Furtskeg, Stout')
    elif text == "похвали пиво":
        bot.send_message(message.chat.id, 'Пиво - это жидкое золото.')
    elif text == "хочу сладкое пиво":
        bot.send_message(message.chat.id,
                         'Reeds, Dr.Diesel, Chester, Кулинария!!! Выбираешь, что хочешь - просто мед!!!')
    elif text == "хочу кислое пиво":
        bot.send_message(message.chat.id,
                         'Заходишь  в магазин и берешь любое с надписью  SOUR! Ну а по фирмам - Panzer, Kwak, '
                         'Rodenbach.')
    elif text == "хочу горькое пиво":
        color_handler(message)
    elif text == "40 - 50 рублей":
        bot.send_message(message.chat.id, "бери жигулевское, братанчик")
    elif text == "50 - 300 рублей":
        bot.send_message(message.chat.id, "ну можно взять  Heieken, Bear Beer или Krusovice")
    elif text == "> 300 рублей":
        taste_handler(message)
    elif text == "gif":
        link = get_gif_by_name(message)
        bot.send_animation(message.from_user.id, link)
    else:
        bot.send_message(message.chat.id, "Я не понимаю о чём вы")


bot.polling(none_stop=True, interval=0)


ModuleNotFoundError: No module named 'telebot'